In [ ]:
import nltk
import torch
import tensorflow as tf
import keras
import warnings
warnings.simplefilter('ignore')

In [ ]:
!pip3 install -U textblob
!pip3 install -U rake-nltk
!python3 -m textblob.download_corpora

In [ ]:
text = []

In [ ]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [ ]:
def preprocess(txt): #convert to lower case, remove stopwords, get important entities
    
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize as tokenize
    
    stop_words=set(stopwords.words('english'))
    t_list, key_phrases, entities = [],[],[]
    txt_tokens=tokenize(txt)
    
    t_list = [word.lower() for word in txt_tokens if not word in stop_words]
    print("After removing stopwords :\n",t_list) #return this
    
    pre_processed_string = ""
    for word in t_list :
        pre_processed_string=pre_processed_string+word+" "
    pre_processed_string.strip()
    print("Preprocessed String : \n",pre_processed_string)
    
    from rake_nltk import Rake
    
    r=Rake()
    r.extract_keywords_from_text(txt)
    key_phrases = r.get_ranked_phrases()
    print("Key Phrases : \n",key_phrases) #return this
    
    import spacy
    from spacy import displacy
    from collections import Counter
    import en_core_web_sm
    nlp = en_core_web_sm.load()
    
    doc = nlp(txt)
    entities = [(X.text, X.label_) for X in doc.ents]
    if len(entities) != 0:
        print("Entities : \n",entities) #return this
    else:
        print("No Entities")
    

In [ ]:
def information_from_article(url):
    
    import spacy
    import requests
    import re
    from bs4 import BeautifulSoup
    
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))


In [ ]:
def get_information(url):
    
    import warnings
    warnings.simplefilter('ignore')
    extractor=information_from_article(url)
    article = nlp(extractor)
    print("Number of entities in the page : ",len(article.ents)) #return this
    labels = [x.label_ for x in article.ents]
    
    i = len(labels)//3 #30% of the top label count
    items = [x.text for x in article.ents]
    common_items = [] #Counter(items).most_common(i)
    too_common=[x for x in range(5)]
    for item in Counter(items).most_common(i):
        try:
            if int(str(item).split(',')[1].split(')')[0]) in too_common:
                continue
            else:
                common_items.append(str(item).split(',')[0])
        except ValueError as e:
            pass
        
    sentences = [x for x in article.sents]
    print(len(sentences))
    print("Most important terms in the document : \n",common_items) #return this
    
    for i in range(len(sentences)):
        displacy.render(nlp(str(sentences[i])), jupyter=True, style='ent')


In [ ]:
get_information("https://medium.com/@jonathan_hui/gan-why-it-is-so-hard-to-train-generative-advisory-networks-819a86b3750b")

In [ ]:
preprocess("THe foo d here is amazing expecially the maggi")